## Importing main libraries

## Conservare un dataset solo per il test, implementare gridsearch seanza sare ealry stopping. Refittare solo il modello migliore con ealy stopping successivamente su tutto il training set e valutarlo sul test set messo da parte all'inizio. Mettere tutti i modelli in un unico file

In [2]:
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
from tensorflow import keras

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


## Reading the dataset

- df_ok: labels of ok gauges
- df_delta: labels of delta gauges
- df_ raw: raw data cointaining the full reading of each gauge

In [3]:
df_delta = pd.read_csv('df_delta.csv') # cointains labels for delta VG
df_ok = pd.read_csv('df_ok.csv')  #contains labels for ok VG

df_raw = pd.read_csv('df_raw.csv') ## contains full reading of each VG

df_labels = pd.concat([df_ok, df_delta], sort=False, axis=0)


df_VG = pd.merge(df_raw, df_labels, on =['gauge_id','fillNumber'])
df_VG = df_VG.set_index(['gauge_id','fillNumber'], drop=True)

## Removing categorical values
df_VG.y.replace(to_replace=['ok', 'delta'], value=[0, 1], inplace=True)

## Splitting data

In [16]:
from sklearn.model_selection import train_test_split

X = np.array(df_VG.iloc[:, :-1])
y = np.array(df_VG.iloc[:, -1])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(194, 3000)
(84, 3000)
(194,)
(84,)


## Defining scalers for CNN

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, RobustScaler


class RowStandardScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        scaler = StandardScaler()
        return scaler.fit_transform(X.transpose()).transpose()

scaler = RowStandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]

In [24]:
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=15, kernel_size=20, activation='relu', padding='same', use_bias=True, input_shape=(3000, 1)),
    keras.layers.MaxPool1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=100, activation='relu'),
    keras.layers.Dense(units=1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
                        optimizer=keras.optimizers.Adam(),
                        metrics=[keras.metrics.Recall()])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 3000, 15)          315       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1500, 15)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 22500)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               2250100   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 2,250,516
Trainable params: 2,250,516
Non-trainable params: 0
_________________________________________________________________


In [26]:
history =  model.fit(X_train, y_train, epochs=100,
                     validation_data=(X_valid, y_valid),
                     callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)] )

Train on 194 samples, validate on 84 samples
Epoch 1/100
194/194 [==============================] - 1s 3ms/sample - loss: 2.3573e-05 - recall: 1.0000 - val_loss: 0.4402 - val_recall: 0.9333
Epoch 2/100
194/194 [==============================] - 1s 3ms/sample - loss: 2.3235e-05 - recall: 1.0000 - val_loss: 0.4407 - val_recall: 0.9333
Epoch 3/100
194/194 [==============================] - 1s 3ms/sample - loss: 2.2887e-05 - recall: 1.0000 - val_loss: 0.4410 - val_recall: 0.9333
Epoch 4/100
194/194 [==============================] - 1s 3ms/sample - loss: 2.2563e-05 - recall: 1.0000 - val_loss: 0.4415 - val_recall: 0.9333
Epoch 5/100
194/194 [==============================] - 1s 3ms/sample - loss: 2.2343e-05 - recall: 1.0000 - val_loss: 0.4419 - val_recall: 0.9333
Epoch 6/100
194/194 [==============================] - 1s 3ms/sample - loss: 2.2059e-05 - recall: 1.0000 - val_loss: 0.4425 - val_recall: 0.9333


In [28]:
y_pred = model.predict(X_valid)

In [34]:
print(np.round(y_pred[:,0])[:10])
print(y_valid[:10])

[0. 1. 1. 0. 0. 0. 1. 1. 0. 0.]
[0 1 1 0 0 0 1 1 0 0]


In [39]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true=y_valid, y_pred=np.round(y_pred[:,0]), sample_weight=None)

0.9404761904761905